In [1]:
import numpy as np
import random
from typing import Callable, Union
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
class Neuron:
    def __init__(self):
        self.weights = None

    def compile(self, weights_sz:int):
        self.weights = np.array([random.uniform(0, 10) for _ in range(weights_sz)]).reshape(weights_sz, 1)

In [28]:
class Layer:
    def sigmoid(self, x:float)-> float:
        return 1/(1+np.exp(-x))
    
    def linearfctn(self, x:float) -> float:
        return x
    
    def softmax(self, x:np.array) -> np.array:
        return np.exp(x)/np.sum(np.exp(x))
    
    def __init__(self, neurons:int, act_fctn:Union[str| Callable]=None, name:str=None):
        self.name = name
        if act_fctn is None:
            self.actn_fn = self.linearfctn
        else:
            self.actn_fn = self.sigmoid if act_fctn=="sigmoid" else self.softmax
            
        self.neurons = [Neuron() for _ in range(neurons)]

    def __str__(self):
        return f"Layer : {self.name}, activation function : {self.actn_fn}, No. of neurons : {len(self.neurons)}"

    def compile(self, childs:int=None):
        for i in self.neurons:
            i.compile(childs)
        self.weights = [neuron.weights for neuron in self.neurons]
        return len(self.neurons)
    
    def forward(self, inp:np.array):
        z = inp @ np.array(self.weights).squeeze().T
        self.temp_output = self.actn_fn(z)
        return self.temp_output
    
    def backward(self, inp:np.array, out:np.array, lr:float=0.01, prev_back:np.array=None):
        if self.actn_fn == self.softmax:
            dw = self.temp_output - out
            dwdx = dw @ inp
            self.weights = self.weights - lr * dwdx
        elif self.actn_fn == self.sigmoid:
            dsig = self.temp_output * (1-self.temp_output)
            dsigdw = dsig * inp
            self.weights -= (lr * dsigdw)

In [29]:
class Tangrad:    
    def __init__(self, layer:list[Layer]=[], inp_features:int=None):
        self.layer = layer
        self.inp_features = inp_features
    
    def add_layer(self, layer:Layer):
        self.layer.append(layer)
    
    def compile(self):
        compiled = 0
        for i in range(len(self.layer)):
            if i == 0:
                compiled = self.layer[i].compile(self.inp_features)
                print(self.layer[i])
            else:
                compiled = self.layer[i].compile(compiled)
                print(self.layer[i])
        
    def fit(self, inp:np.array, out:np.array, lr:float=0.01):
        for i in self.layer:
            inp = i.forward(inp)
        # for i in range(len(self.layer),-1,-1):
        #     self.layer[i].backward(inp, out, lr)
        return inp

In [30]:
Ann = Tangrad(inp_features=4)

In [31]:
layer_0 = Layer(4, "sigmoid","layer 0")
layer_1 = Layer(20, "sigmoid","layer 1")
layer_2 = Layer(10, "sigmoid","layer 2")
layer_3 = Layer(1, "sigmoid","layer 3")
Ann.add_layer(layer_0)
Ann.add_layer(layer_1)
Ann.add_layer(layer_2)
Ann.add_layer(layer_3)

In [32]:
Ann.compile()

Layer : layer 0, activation function : <bound method Layer.sigmoid of <__main__.Layer object at 0x7374073f4ad0>>, No. of neurons : 4
Layer : layer 1, activation function : <bound method Layer.sigmoid of <__main__.Layer object at 0x737407432e90>>, No. of neurons : 20
Layer : layer 2, activation function : <bound method Layer.sigmoid of <__main__.Layer object at 0x737407432fd0>>, No. of neurons : 10
Layer : layer 3, activation function : <bound method Layer.sigmoid of <__main__.Layer object at 0x73740742b490>>, No. of neurons : 1


In [33]:
iris  = load_iris()

In [34]:
x = iris.data[:, :]
y = iris.target_names[iris.target] == "virginica"
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=85)

In [35]:
scaler = StandardScaler()
traindata = scaler.fit_transform(x_train)
testdata = scaler.fit_transform(x_test)

In [36]:
Ann.fit(traindata, y_train)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])